In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import Adam
from torchvision.transforms import transforms, Compose, ToTensor, Lambda
from torch.utils.data import DataLoader, random_split
from tools.models import *
from tools.data_utils import *
import matplotlib.pyplot as plt
import numpy as np
import random
import imageio
from argparse import ArgumentParser
from tools.plot_utils import show_images, show_forward, generate_new_images
from tools.models import Autoencoder
from torch.utils.data import Subset
from collections import OrderedDict
import json
import tools.sr3 as sr3
# from misc.print_diffuse_feats import print_feats

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
opt = {
    "name": "ddpm-RS-CDHead-LEVIR",
    "phase": "test", 
    "gpu_ids": [
        0
    ],
     "path": {
        "resume_state": "./history/pretrained_sr3/sr3_50_100"
    },

    "datasets": {
        "train": {
            "name": "LEVIR-CD-256",
            "dataroot": "dataset/LEVIR-CD256/",
            "resolution": 256, 
            "batch_size": 8,
            "num_workers": 8,
            "use_shuffle": True,
            "data_len": -1 
        },
        "val": {
            "name": "LEVIR-CD-256",
            "dataroot": "dataset/LEVIR-CD256/",
            "resolution": 256, 
            "batch_size": 4,
            "num_workers": 8,
            "use_shuffle": True,
            "data_len": -1 
        },
        "test": {
            "name": "LEVIR-CD-256",
            "dataroot": "dataset/LEVIR-CD256/",
            "resolution": 256, 
            "batch_size": 4,
            "num_workers": 8,
            "use_shuffle": False,
            "data_len": -1 
        }
    },
    "model_cd": {
        "feat_scales": [2, 5, 8, 11, 14],
        "out_channels": 2,
        "loss_type": "ce",
        "output_cm_size": 256,
        "feat_type": "dec", 
        "t": [50, 100]
    },

    "model": {
        "which_model_G": "sr3", 
        "finetune_norm": False,
        "unet": {
            "in_channel": 3,
            "out_channel": 3,
            "inner_channel": 128,
            "channel_multiplier": [
                1,
                2,
                4,
                8,
                8
            ],
            "attn_res": [
                16
            ],
            "res_blocks": 2,
            "dropout": 0.2
        },
        "beta_schedule": { 
            "train": {
                "schedule": "linear",
                "n_timestep": 2000,
                "linear_start": 1e-6,
                "linear_end": 1e-2
            },
            "val": {
                "schedule": "linear",
                "n_timestep": 2000,
                "linear_start": 1e-6,
                "linear_end": 1e-2
            },
            "test": {
                "schedule": "linear",
                "n_timestep": 2000,
                "linear_start": 1e-6,
                "linear_end": 1e-2
            }
        },
        "diffusion": {
            "image_size": 256,
            "channels": 3, 
            "loss": "l2", 
            "conditional": False 
        }
    }
}

In [3]:
# Hyperparameters
dataset = "Building"
store_path = f"./history/ddpm_model_{dataset}.pt"
batch_size = 16
input_channel = 3
n = 10
resize = Resize256

data_folder = f"random_n_pins_{n}"
transformed_dataset = PinDataset(csv_file=f"./data/{dataset}/processed/{data_folder}/pins.csv",
                             root_dir=f"./data/{dataset}/processed/images/",
                             transform=Compose([ToTensor(), resize(), Lambda()]))
            
data_loader = DataLoader(transformed_dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate_fn)
# if os.path.exists(f"./data/{dataset}/train_indices.npy"):
#     train_indices = np.load(f'./data/{dataset}/train_indices.npy')
#     val_indices = np.load(f'./data/{dataset}/val_indices.npy')
#     test_indices = np.load(f'./data/{dataset}/test_indices.npy')
#     train_indices = np.concatenate((train_indices, np.arange(1000, 1697))) # add all un-used images for DDPM training
#     # Use the indices to create new datasets
#     train_dataset = Subset(transformed_dataset, train_indices)
#     val_dataset = Subset(transformed_dataset, val_indices)
#     test_dataset = Subset(transformed_dataset, test_indices)
# else:
#     # Split the dataset into train, validation, and test sets
#     train_dataset, val_dataset, test_dataset = random_split(
#         transformed_dataset, [train_size, val_size, test_size]
#     )
#     np.save(f'./data/{dataset}/train_indices.npy', train_dataset.indices)
#     np.save(f'./data/{dataset}/val_indices.npy', val_dataset.indices)
#     np.save(f'./data/{dataset}/test_indices.npy', test_dataset.indices)

# Create your DataLoader with the custom_collate_fn
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate_fn)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate_fn)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=custom_collate_fn)

NameError: name 'Resize256' is not defined

In [4]:
def concat_feature_maps(feature_maps, layers=[5, 6, 7, 8]):
    # Define empty list to store upsampled feature maps for specific layers
    upsampled_maps = []
    
    # Upsample and store feature maps for specified layers
    for t in range(len(f_A)):
        for layer_idx in layers:
            fmap = feature_maps[t][layer_idx]
            upsampled_fmap = torch.nn.functional.interpolate(fmap, size=(128, 128), mode='bilinear', align_corners=False)
            upsampled_maps.append(upsampled_fmap)
    
    # Concatenate the upsampled feature maps along the channel dimension
    concatenated_maps = torch.cat(upsampled_maps, dim=1)
    
    return concatenated_maps

def save_fm_by_batch(opt, data_loader, images_directory, output_directory):
    f_A = []
    opt = sr3.dict_to_nonedict(opt)
    # Loading diffusion model
    diffusion = sr3.DDPM(opt)
    # Set noise schedule for the diffusion model
    diffusion.set_new_noise_schedule(
        opt['model']['beta_schedule'][opt['phase']], schedule_phase=opt['phase'])   

    # Create the output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)
    images_directory = os.path.join(images_directory, 'images')

    # Create subdirectories for images and count labels
    os.makedirs(images_directory, exist_ok=True)

    # Save images as "0.png" or "0.npy", "1.png" or "1.npy", etc., and dump data to CSV
    with open(os.path.join(output_directory, 'pins.csv'), 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)

        # Write the header row
        csv_writer.writerow(['image', 'pins', 'outputs'])

        for batch_idx, data in enumerate(data_loader):
            images = batch['image'].to(device) # get RGB instead of RGBA
            pins = batch['pins']
            outputs = batch['outputs']
            diffusion.feed_data(train_data)   
            for t in opt['model_cd']['t']:
                fe_A_t, fd_A_t= diffusion.get_feats(t=t) #np.random.randint(low=2, high=8)
                f_A.append(fd_A_t)
            concat_fm = concat_feature_maps(f_A)
            
            for i in range(len(images)):
                # Calculate the overall index
                overall_index = batch_idx * len(batch_images) + i

                # Save the image as "overall_index.png" or "overall_index.npy" in the images subdirectory
                image_filename = os.path.join(images_directory, f"{overall_index}")
                # For multi-channel images, save as NPY
                image_filename += ".npy"
                if not os.path.exists(image_filename):
                    np.save(image_filename, concat_fm[i].detach().cpu().numpy())

                # Write data to CSV
                csv_writer.writerow([os.path.basename(image_filename), pins[i], outputs[i]])

    print("Data and images have been saved to the CSV and image files.")

In [5]:
save_fm_by_batch(opt, data_loader, images_directory="./data/Building_ddpm/images", output_directory=f"./data/Building_ddpm/{data_folder}")

NameError: name 'data_loader' is not defined

In [ ]:
for idx in range(0, opt['datasets']['val']['data_len']):
    diffusion.test(in_channels=3, img_size=256, continous=True)
    visuals = diffusion.get_current_visuals()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

count = 0
for batch in train_loader:
    images = batch['image'].to(device) # get RGB instead of RGBA
    pins = batch['pins']
    outputs = batch['outputs']
    count += 1

In [ ]:
# Loading the trained model
store_path = f"./history/ddpm_model_{dataset}.pt"
best_model = DDPM(UNet(), n_steps=n_steps, device=device)
best_model.load_state_dict(torch.load(store_path, map_location=device))
best_model.eval()
print("Model loaded")

print("Generating new images")
generated = generate_new_images(
        best_model,
        n_samples=100,
        device=device,
        gif_name="mnist.gif"
    )
show_images(generated, "Final result")

In [ ]:
show_forward(model, train_dataloader, device)

In [ ]:
# Visual evaluation on the test set

# change for dataset
num_kernels_encoder = [16, 8]
num_kernels_decoder = [16]
input_channel = 1

test_dataloader = DataLoader(dataset, batch_size, shuffle=True)

with torch.no_grad():
    for data in test_dataloader:
        inputs, _ = data
        inputs = inputs.view(inputs.size(0), -1)
        
        # Forward pass through the autoencoder
        autoencoder = Autoencoder(num_kernels_encoder, num_kernels_decoder, input_channel=input_channel).to(device)
        outputs = autoencoder(inputs)
        
        # Reshape the outputs to the original image shape (28x28)
        outputs = outputs.view(outputs.size(0), 1, 28, 28)
        
        # Convert Tensors to NumPy arrays for visualization
        inputs = inputs.view(inputs.size(0), 1, 28, 28).numpy()
        outputs = outputs.numpy()
        
        # Plot the input and output images
        for i in range(inputs.shape[0]):
            plt.figure(figsize=(8, 4))
            plt.subplot(1, 2, 1)
            plt.title('Input Image')
            plt.imshow(np.squeeze(inputs[i]), cmap='gray')
            plt.axis('off')
            
            plt.subplot(1, 2, 2)
            plt.title('Output Image')
            plt.imshow(np.squeeze(outputs[i]), cmap='gray')
            plt.axis('off')
            plt.show()